In [1]:
import wandb
from ultralytics import YOLO
import torch
from torch.quantization import quantize_dynamic

In [2]:
#pip install wandb

In [3]:
#pip install ultralytics

In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용하도록 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # 특정 GPU에 연산을 할당
        tf.config.set_visible_devices(gpus[0], 'GPU')
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPU


In [6]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "yolo_quan_local.ipynb"  # 노트북 이름을 설정하세요.

In [7]:
import wandb

# WandB에 로그인
wandb.login(key='debec7acbfa814ac1d35013a84ac13808d992cd0')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\희정\_netrc


True

In [8]:
# # sweep 구성설정
# sweep_config = {
#     'name': 'yolo_sweep',
#     'method': 'random',  # 또는 "grid"를 사용할 수 있습니다.
#     'metric': {'name': 'val_map50', 'goal': 'maximize'},
#     'parameters': {
#         'batch_size': {'values': [16, 32, 64]},
#         'learning_rate': {'min': 1e-5, 'max': 1e-2},
#         'epochs': {'values': [10, 20, 30, 40, 50]}
#     }
# }

# wandb Sweep 설정 - 최적화할 metric을 box_loss로 설정
sweep_config = {
    "method": "bayes",  # Bayesian optimization으로 하이퍼파라미터 탐색
    "metric": {"name": "box_loss", "goal": "minimize"},
    "parameters": {
        "lr": {"values": [0.001, 0.0005, 0.0001]},
        "epochs": {"values": [10, 20]},
        "batch_size": {"values": [16, 32]},
        "patience": {"value": 3}  # Early stopping을 위한 patience 추가
    },
}
sweep_id = wandb.sweep(sweep_config, project="yolo11-quantization")

In [9]:
# # Sweep ID 생성
# sweep_id = wandb.sweep(sweep_config, project="yolo11-quantization")

Create sweep with ID: r4oeedyv
Sweep URL: https://wandb.ai/icejoy000524-student/yolo11-quantization/sweeps/r4oeedyv


In [14]:
def train():
    # WandB 초기화
    wandb.init(project="yolo11-quantization", entity="icejoy000524-student")

    # wandb Sweep으로 설정된 하이퍼파라미터 불러오기
    config = wandb.config
    learning_rate = config.lr
    epochs = config.epochs
    batch_size = config.batch_size
    patience = config.patience  # Early stopping을 위한 patience 불러오기

    # YOLOv11 모델 로드
    model = YOLO("yolo11n.pt")  # YOLOv11 모델 가중치 파일 경로

    best_mAP = 0
    epochs_without_improvement = 0

    # 훈련 설정
    results = model.train(
        data="C:/Users/희정/NIPAPROJECT/yolodata/yolo_split/yolo.yaml",  # 데이터셋 YAML 파일 경로
        epochs=epochs,  # Sweep에서 정의한 에폭 수
        imgsz=640,  # 이미지 크기
        batch=batch_size,  # Sweep에서 정의한 배치 사이즈
        lr0=learning_rate,
        device="cuda" if torch.cuda.is_available() else "cpu",  # 사용 가능한 장치
    )

    # box_loss와 다른 메트릭을 로깅하여 wandb에 기록
    for epoch, metrics in enumerate(results):
        wandb.log({
            "epoch": epoch,
            "mAP_0.5": metrics["metrics/mAP_0.5"],
            "box_loss": metrics["loss/box"],  # 최적화할 손실 메트릭
            "obj_loss": metrics["loss/obj"],
            "cls_loss": metrics["loss/cls"],
        })

        # Early Stopping 로직
        current_mAP = metrics["metrics/mAP_0.5"]
        if current_mAP > best_mAP:
            best_mAP = current_mAP
            epochs_without_improvement = 0  # 개선이 있었으므로 카운트 초기화
            wandb.log({"best_mAP": best_mAP})  # 최상의 mAP 기록
        else:
            epochs_without_improvement += 1
        
        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch}")
            break  # 조기 중단

In [15]:
# Sweep 에이전트 실행
wandb.agent(sweep_id, function=train, count=10)

wandb: Agent Starting Run: z80ngibh with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	learning_rate: 0.008071457081693233
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Ultralytics 8.3.25  Python-3.8.19 torch-2.4.1+cpu CPU (Intel Core(TM) i7-10700 2.90GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users//NIPAPROJECT/yolodata/yolo_split/yolo.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 11.9MB/s]


Overriding model.yaml nc=80 with nc=24

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train...:   0%|          | 0/752 [00:00<?, ?it/s]Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 89 images, 0 backgrounds, 0 corrupt:  12%|█▏    Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 220 images, 0 backgrounds, 0 corrupt:  29%|██▉  Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 351 images, 0 backgrounds, 0 corrupt:  47%|████▋Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 485 images, 0 backgrounds, 0 corrupt:  64%|█████Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 615 images, 0 backgrounds, 0 corrupt:  82%|█████Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 746 images, 0 backgrounds, 0 corrupt:  99%|█████Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\train... 752 images, 0 backgrounds, 0 corrupt: 100%|█████


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<0


                   all         94        801      0.532      0.291      0.348      0.205
                 bench          1          2          1          0          0          0
                   car         76        261      0.664      0.858      0.866      0.626
          potted_plant          9         26      0.386      0.308        0.3      0.125
          fire_hydrant          3          4          1          0     0.0273     0.0154
       movable_signage         14         27      0.343       0.37      0.395      0.239
            motorcycle          4          4      0.327       0.25      0.321      0.244
              stroller          2          2          1          0     0.0138     0.0124
                 table          2          2          0          0     0.0111    0.00221
                   bus          7          9      0.278      0.111      0.215      0.109
               bicycle          4          7       0.71      0.857      0.795      0.333
          traffic_sig

val: Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\val.cache... 94 images, 0 backgrounds, 0 corrupt: 100%|████Scanning C:\Users\희정\NIPAPROJECT\yolodata\yolo_split\val.cache... 94 images, 0 backgrounds, 0 corrupt: 100%|████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:05<0


                   all         94        801      0.532      0.291      0.348      0.205
                 bench          1          2          1          0          0          0
                   car         76        261      0.664      0.858      0.866      0.626
          potted_plant          9         26      0.386      0.308        0.3      0.125
          fire_hydrant          3          4          1          0     0.0273     0.0154
       movable_signage         14         27      0.343       0.37      0.395      0.239
            motorcycle          4          4      0.327       0.25      0.321      0.244
              stroller          2          2          1          0     0.0138     0.0124
                 table          2          2          0          0     0.0111    0.00221
                   bus          7          9      0.278      0.111      0.215      0.109
               bicycle          4          7       0.71      0.857      0.795      0.333
          traffic_sig

quantized_model_size,▁
val_fitness,▁
val_map,▁
val_map50,▁
val_map50_95,▁
val_precision,▁
val_recall,▁
quantized_model_path,yolo11_quantized.pt
quantized_model_size,2586832
val_fitness,0.21971
val_map,0.20543
